In [1]:
import numpy as np
import pandas as pd
import re
import tensorflow as tf
import nltk
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import torch

print("PyTorch 버전:", torch.__version__)
print("CUDA 사용 가능 여부:", torch.cuda.is_available())
print("PyTorch에서 사용하는 CUDA 버전:", torch.version.cuda)
torch.set_default_device("cuda")
# GPU용 Generator 생성
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = torch.Generator(device=device)

PyTorch 버전: 2.5.1+cu118
CUDA 사용 가능 여부: True
PyTorch에서 사용하는 CUDA 버전: 11.8


In [2]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
state_dict = torch.load("parameter/final_model_weights5.pth", weights_only=True)  # 가중치 로드
model.load_state_dict(state_dict)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [3]:
data = pd.read_csv("cleaned_data.csv", sep=';', on_bad_lines='skip')
texts = data['text'].tolist()
labels = data['type'].tolist()
#학습용, 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)
# BERT 토크나이저
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [4]:
torch.cuda.empty_cache()  # PyTorch가 사용하지 않는 메모리 반환

# 평가
model.eval()
test_texts = X_test
labels = y_test
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128, return_tensors="pt")
total = len(test_texts)

# 데이터셋 정의
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

batch_size=32

test_dataset = CustomDataset(test_encodings, labels)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

correct = 0
    
with torch.no_grad():
    for batch in test_loader:
        # 배치 데이터 가져오기
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        
        # 모델 예측
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits # [batch_size, 1] -> [batch_size]
        # 소프트맥스 확률 계산
        probabilities = torch.softmax(logits, dim=-1)  # [batch_size, num_classes]
        # 가장 큰 확률의 클래스 인덱스를 예측
        predictions = torch.argmax(probabilities, dim=-1)

        # 정확도 계산
        correct += (predictions == labels.int()).sum().item()
        
print("Accuracy: ", correct/total)

C:\Users\wwir9\anaconda3\envs\cnn\lib\site-packages\torch\utils\_device.py:106: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)


Accuracy:  0.9212818128012176


In [15]:
print("Accuracy:  0.8745243933372792")
print("Accuracy:  0.927369578084045")
print("Accuracy:  0.9276232349708294")
print("Accuracy:  0.9274541303796398")
print("Accuracy:  0.9247484569206054")
print("Accuracy:  0.9212818128012176")

Accuracy:  0.8745243933372792
Accuracy:  0.927369578084045
Accuracy:  0.9276232349708294
Accuracy:  0.9274541303796398
Accuracy:  0.9247484569206054
Accuracy:  0.9212818128012176
